# Generación de Texto Con GPT-2

In [1]:
pip install transformers torch --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers adapter-transformers --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install peft transformers --quiet

Note: you may need to restart the kernel to use updated packages.


**Preprocesamiento y tokenización del corpus**

In [26]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from torch.utils.data import Dataset
import textwrap

# Cargar el tokenizador y el modelo GPT-2 en español
tokenizer = GPT2Tokenizer.from_pretrained("datificate/gpt2-small-spanish")
model = GPT2LMHeadModel.from_pretrained("datificate/gpt2-small-spanish")

# Agregar el token de padding
tokenizer.pad_token = tokenizer.eos_token

# Leer el corpus completo
with open("corpus_argentina_limpio.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Tokenizar en fragmentos de longitud fija
chunk_size = 512
tokens = []

for i in range(0, len(text), chunk_size):
    chunk = text[i:i + chunk_size]
    tokenized_chunk = tokenizer(chunk, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    tokens.append(tokenized_chunk["input_ids"])

# Concatenar los fragmentos
input_ids = torch.cat(tokens, dim=0)

**Entrenamiento de GPT-2 con el corpus de texto**

In [27]:
class TextDataset(Dataset):
    def __init__(self, tokens):
        self.tokens = tokens

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        return {
            "input_ids": self.tokens[idx],
            "attention_mask": torch.ones_like(self.tokens[idx]),
            "labels": self.tokens[idx]
        }

train_dataset = TextDataset(tokens)

# Configuración de argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./gpt2-spanish-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=2,
    save_steps=1000,
    save_total_limit=2,
    prediction_loss_only=True,
    learning_rate=1e-5,
    report_to="none"
)

# Entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Iniciar el entrenamiento completo
trainer.train()

  0%|          | 0/240 [00:00<?, ?it/s]

{'train_runtime': 162.9608, 'train_samples_per_second': 2.945, 'train_steps_per_second': 1.473, 'train_loss': 3.7096323649088543, 'epoch': 10.0}


TrainOutput(global_step=240, training_loss=3.7096323649088543, metrics={'train_runtime': 162.9608, 'train_samples_per_second': 2.945, 'train_steps_per_second': 1.473, 'total_flos': 125420175360000.0, 'train_loss': 3.7096323649088543, 'epoch': 10.0})

In [28]:
# Configuración del modelo y tokenizador en modo evaluación
model.eval()

# Función para generar texto
def generate_text(prompt, min_length=40, max_length=60):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            min_length=min_length,
            eos_token_id=tokenizer.eos_token_id,  
            no_repeat_ngram_size=2,
            top_k=30,
            top_p=0.85,
            temperature=0.7,
            do_sample=True,
            early_stopping=True
        )
        
    # Decodificar el texto generado
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Asegurar que termine con un punto
    if not generated_text.endswith('.'):
        generated_text = generated_text.rstrip() + '.'
        
    return generated_text

# Texto de inicio
prompt_text = "la selección argentina"

# Generación de texto
generated_text = generate_text(prompt_text)
print("Texto generado:\n", textwrap.fill(generated_text, width=70))

/Users/achula/miniconda3/envs/cell_detection/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:649: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Texto generado:
 la selección argentina ha sido una de las mejores del torneo. El
partido fue muy bueno y el equipo ganó el partido. Pero el último
partido es importante, ya que es el momento más importante para la
selección, porque es la última oportunidad de hacer que el mundo salga
de la pobreza. Y el final.
